<a href="https://colab.research.google.com/github/sonii26/API/blob/main/Implement_text_to_text_generation_model_for_polishing_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets -q


In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-small"  # lightweight and fast
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
def polish_text(text):
    input_text = "polish: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)

    output_ids = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
    polished = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return polished


In [4]:
rough_input = "i wants to go school"
print("Original:", rough_input)
print("Polished:", polish_text(rough_input))


Original: i wants to go school
Polished: polish polish: i wants to go school


FastAPI

In [5]:
!pip install transformers fastapi uvicorn pyngrok -q


In [6]:
!pip install transformers fastapi "uvicorn[standard]" nest-asyncio pyngrok -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 kB 13.6 MB/s eta 0:00:00


In [7]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import threading

# Load T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


 the polishing function

In [8]:
def polish_text(text):
    input_text = "polish: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)
    output_ids = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
    polished = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return polished


In [9]:
app = FastAPI()

class Query(BaseModel):
    text: str

@app.post("/polish")
async def polish(query: Query):
    result = polish_text(query.text)
    return {"original": query.text, "polished": result}


In [15]:
# Enable nested event loop for running FastAPI inside Jupyter/Colab
nest_asyncio.apply()

# Open a tunnel on port 8000
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Run FastAPI in a separate thread so notebook doesn't block
thread = threading.Thread(target=run, daemon=True)
thread.start()


Public URL: NgrokTunnel: "https://ef74-34-74-109-235.ngrok-free.app" -> "http://localhost:8000"


Exception in thread Thread-10 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-15-143534778.py", line 9, in run
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 40, in _get_event_loop
    loop = events.get_event_loop_policy().get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/py

In [12]:
from pyngrok import ngrok

# Paste your auth token string here:
NGROK_AUTH_TOKEN = "2yShxu6XhJhYAiXxl1XO6VVPKgL_72tfC1wDAqGhDDWj7J8PY"

# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


In [13]:
public_url = ngrok.connect(8000)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://a1e5-34-74-109-235.ngrok-free.app" -> "http://localhost:8000"
